## Trabajo Practico 6: Filtros de Haar

Utilizando la webcam
1. Implementar un algoritmo utilizando filtros de Haar en cascada que:
 * Detecte Rostros
 * Dentro de los rostros detecte ojos
 * Dentro de los rostros detecte sonrisas

In [9]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [56]:
# Leemos los clasificadores pre-entrenados (instalados en cv2/data/haarcascades/)
#--------------------------------------------------------------------------------
face_cascade = cv.CascadeClassifier('haarcascades\haarcascade_frontalface_default.xml')

eye_cascade = cv.CascadeClassifier('haarcascades\haarcascade_eye.xml')
eye_cascade_eyeclasses = cv.CascadeClassifier('haarcascades\haarcascade_eye_tree_eyeglasses.xml')

smile_cascade = cv.CascadeClassifier('haarcascades\haarcascade_smile.xml')

In [140]:
cap = cv.VideoCapture(0)

fourcc = cv.VideoWriter_fourcc(*'MP4V')
video = cv.VideoWriter('FiltroHaar_TP6.mp4', fourcc, 20.0, (640,480))

if(cap.isOpened()==False):
    print("Error opening camera")
    
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
               
        # Pasamos la imagen a escala de grises
        #-------------------------------------
        grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        grayFrame = cv.equalizeHist(grayFrame)
        
        # Llamamos al clasificador de Haar (AdaBoost)
        #--------------------------------------------
        #faces = face_cascade.detectMultiScale(grayImage, 1.012, 5)
        faces = face_cascade.detectMultiScale(grayFrame, 1.012, 5, minSize= (100,130),maxSize=(400,600))

        # Recorro las caras encontradas
        #------------------------------
        for (x,y,w,h) in faces:
            # Le dibujamos un rectángulo amarillo
            cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,250),2)
            # Definimos las ROIs en la imagen gris y color
            roi_gray = grayFrame[y:y+h, x:x+w] 
            roi_color = frame[y:y+h, x:x+w] 
            # Para cada rostro hallado le buscamos los ojos
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.02, 5, minSize= (5,5),maxSize=(np.uint32((x+w)/4),np.uint32((y+h)/4)))
            # En los ojos hallados les dibujamos rectángulos
            for (ex,ey,ew,eh) in eyes:
                cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
            # Para cada rostro hallado le buscamos una sonrisa
            #roi_smile_gray = grayFrame[y+eh+ey:y+h, x:x+w]
            #roi_smile_color = frame[y+eh+ey:y+h, x:x+w]
            roi_smile_gray = grayFrame[np.uint(y+h/2):y+h, x:x+w]
            roi_smile_color = frame[np.uint(y+h/2):y+h, x:x+w]
            smile = smile_cascade.detectMultiScale(roi_smile_gray, 1.08, 6, minSize= (np.uint32((x+w)/6),np.uint32((y+h)/6)),maxSize=(np.uint32((x+w)/2),np.uint32((y+h)/2)))
            # En los ojos hallados les dibujamos rectángulos
            for (sx,sy,sw,sh) in smile:
                cv.rectangle(roi_smile_color,(sx,sy),(sx+sw,sy+sh),(255,255,0),2)
        
        
        cv.imshow('Frame', frame)
        video.write(frame)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

cv.destroyAllWindows()
video.release()
cap.release()

cv.destroyAllWindows()